### Combine all of the files to upload to the server - will have to do this for snake dataset

## We want to split the data to get it ready to train the model
##### Remember to open the notebook within the conda environment created!!!!!! cd to environment and "jupyter notebook" to open

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
import shutil
import math
import imutils

In [ ]:
SEED_DATA_DIR = "C:\TutorialBuild\BrainTumor"
num_of_images = {}

In [ ]:
for dir in os.listdir(SEED_DATA_DIR):
    num_of_images[dir] = len(os.listdir(os.path.join(SEED_DATA_DIR, dir )))

print (num_of_images)

In [ ]:
# now we want to create 3 folders: 70% train data, 15% validation data and 15% test data

TRAIN_DIR =  "C:\TutorialBuild\BrainTumor\\train"
VALIDATE_DIR =  "C:\TutorialBuild\BrainTumor\\validate"
TEST_DIR =  "C:\TutorialBuild\BrainTumor\\test"

In [ ]:
# create the train folder :
if not os.path.exists(TRAIN_DIR):
    os.mkdir(TRAIN_DIR)

    for dir in os.listdir(SEED_DATA_DIR):
        os.makedirs(TRAIN_DIR + "/" + dir)
        print (TRAIN_DIR + "/" + dir)

        for img in np.random.choice(a=os.listdir(os.path.join(SEED_DATA_DIR,dir)) , size= (math.floor(70/100* num_of_images[dir] )-5) , replace=False ):
            O = os.path.join(SEED_DATA_DIR, dir , img)
            print(O)
            D = os.path.join(TRAIN_DIR, dir)
            print(D)
            shutil.copy(O,D)
            os.remove(O)
else:
    print("Train Folder Exists")

In [ ]:
# create the test folder:
if not os.path.exists(TEST_DIR):
    os.mkdir(TEST_DIR)

    for dir in os.listdir(SEED_DATA_DIR):
        os.makedirs(TEST_DIR + "/" + dir)
        print (TEST_DIR + "/" + dir)

        for img in np.random.choice(a=os.listdir(os.path.join(SEED_DATA_DIR,dir)) , size= (math.floor(15/100* num_of_images[dir] )-5) , replace=False ):
            O = os.path.join(SEED_DATA_DIR, dir , img)
            print(O)
            D = os.path.join(TEST_DIR, dir)
            print(D)
            shutil.copy(O,D)
            os.remove(O)
else:
    print("Test Folder Exists")

In [ ]:
# create the validate folder:
if not os.path.exists(VALIDATE_DIR):
    os.mkdir(VALIDATE_DIR)

    for dir in os.listdir(SEED_DATA_DIR):
        os.makedirs(VALIDATE_DIR + "/" + dir)
        print (VALIDATE_DIR + "/" + dir)

        for img in np.random.choice(a=os.listdir(os.path.join(SEED_DATA_DIR,dir)) , size= (math.floor(15/100* num_of_images[dir] )-5) , replace=False ):
            O = os.path.join(SEED_DATA_DIR, dir , img)
            print(O)
            D = os.path.join(VALIDATE_DIR, dir)
            print(D)
            shutil.copy(O,D)
            os.remove(O)
else:
    print("Validate Folder Exists")

# Here  we are building the CNN model

In [ ]:
from matplotlib.pyplot import cla
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, BatchNormalization, MaxPool2D, GlobalAveragePooling2D
from keras.models import Sequential
from keras.preprocessing import image
import keras

In [ ]:
TRAIN_DIR =  "C:\TutorialBuild\BrainTumor\\train"
VALIDATE_DIR =  "C:\TutorialBuild\BrainTumor\\validate"

In [ ]:
# build the CNN model
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape = (224,224,3)    ))

model.add(Conv2D(filters=64, kernel_size=(3,3),  activation='relu' ))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(3,3),  activation='relu' ))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=256, kernel_size=(3,3),  activation='relu' ))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Dropout(rate=0.25))

model.add(Flatten())

model.add(Dense(units=64, activation='relu'))
model.add(Dropout(rate=0.25))

#final layer:
model.add(Dense(units=1, activation='sigmoid'))

model.compile(loss=keras.losses.binary_crossentropy, optimizer='adam', metrics=['accuracy'])

print(model.summary())

In [ ]:
train_datagen = image.ImageDataGenerator(
    zoom_range=0.2 , shear_range=0.2, rescale=1. / 255 , horizontal_flip=True
)

val_datagen = image.ImageDataGenerator( rescale= 1. / 255)

train_data = train_datagen.flow_from_directory(directory=TRAIN_DIR , target_size=(224,224) , batch_size=32 , class_mode='binary')
val_data = val_datagen.flow_from_directory(directory=VALIDATE_DIR, target_size=(224,224), batch_size=32, class_mode='binary')

In [ ]:
# create model check point for the performence of the model

from keras.callbacks import ModelCheckpoint , EarlyStopping

In [ ]:
# lets stop the training if the accuracy is good

es = EarlyStopping(monitor='val_accuracy', min_delta=0.01 , patience=5 , verbose=1 , mode='auto')
mc = ModelCheckpoint(filepath='C:\TutorialBuild\BrainTumor\MyBestModel.h5', monitor='val_accuracy' ,  verbose=1 , mode='auto' , save_best_only=True)


call_back = [es, mc]

hist = model.fit(x=train_data, epochs=30 , verbose=1, validation_data=val_data, callbacks=call_back)

h = hist.history
print('Keys : ', h.keys() )

In [ ]:
#lets plot the accuracy and the loss
#===================================


#accuracy 
plt.plot(h['accuracy'])
plt.plot(h['val_accuracy'], c='red')
plt.title('Accuracy vs. Val Accuracy')
plt.show()

#loss
plt.plot(h['loss'])
plt.plot(h['val_loss'], c='red')
plt.title('loss vs. Val loss')
plt.show()

# Now we want to check the accuracy of the model with the test data. Then we will run predictions on an image

In [ ]:
from locale import normalize
import numpy as np
from keras.models import load_model
from keras.preprocessing import image
import keras
import cv2

In [ ]:
TEST_DIR = "C:\TutorialBuild\BrainTumor\\test"

In [ ]:
test_datagen = image.ImageDataGenerator( rescale= 1. / 255)
test_data = test_datagen.flow_from_directory(directory=TEST_DIR , target_size=(224,224) , batch_size=32 , class_mode='binary')

In [ ]:
# lets print the classes :

print("test_data.class_indices: ", test_data.class_indices)

In [ ]:
#load the saved model :

model = load_model('C:\TutorialBuild\BrainTumor\MyBestModel.h5')

In [ ]:
#print(model.summary() )

acc = model.evaluate(x=test_data)[1]

print(acc)

In [ ]:
# load an image from the test folder 
imagePath = "C:\TutorialBuild\BrainTumor\test\Healthey\Not Cancer  (1523).jpg"
#imagePath = "C:/Python-cannot-upload-to-GitHub/BrainTumor/test/Brain Tumor/Cancer (17).jpg"

img = image.load_img(imagePath,target_size=(224,224))
i = image.img_to_array(img) # convert to array
i = i / 255 # -> normalize to our model
print(i.shape)

input_arr = np.array([i]) # add another dimention 
print(input_arr.shape)

In [ ]:
# run the prediction
predictions = model.predict(input_arr)[0][0]
print(predictions)

In [ ]:
# since it is binary if the result is close to 0 it is Tumor , and if it close to 1 it is healthy
result = round(predictions)
if result == 0 :
    text = 'Has a brain tumor'
else :
    text = "Brain healthy"


print(text)

In [ ]:
imgResult = cv2.imread(imagePath)
font = cv2.FONT_HERSHEY_COMPLEX
cv2.putText(imgResult, text, (0,20), font, 0.8 , (255,0,0),2 )
cv2.imshow('img', imgResult)
cv2.waitKey(0)
cv2.imwrite("C:\TutorialBuild\BrainTumor\predictImage.jpg",imgResult)